In [3]:
from roboflow import Roboflow

# Roboflow APIキーをセット
rf = Roboflow(api_key="Bkn7HMszGVliBmChzz78")

# プロジェクト名とバージョンを指定
project = rf.workspace("valengou").project("structure-inspector")
model = project.version(2).model

loading Roboflow workspace...
loading Roboflow project...


In [13]:
image_path = "test.jpg"  # ここに画像ファイル名を指定

# 推論を実行
prediction = model.predict(image_path, confidence=50, overlap=50).json()

# 結果を出力
print(prediction)

# さらに、推論結果の画像を保存する場合
model.predict(image_path).save("result.jpg")

{'predictions': [{'x': 52, 'y': 44, 'width': 4, 'height': 6, 'confidence': 0.6482348442077637, 'class': 'Out of Plane Stiffener', 'class_id': 3, 'detection_id': '73566d2f-d67f-4b9e-982f-65d787bab00f', 'image_path': 'test.jpg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '177', 'height': '148'}}


In [18]:
from roboflow import Roboflow
import cv2

# Roboflowセットアップ
rf = Roboflow(api_key="Bkn7HMszGVliBmChzz78")
project = rf.workspace("valengou").project("structure-inspector")
model = project.version(2).model

# 画像のパス
image_path = "test2.jpg"

# 推論 (結果をjson形式でもらう)
prediction = model.predict(image_path, confidence=50, overlap=50).json()

# OpenCVで元画像を読む
img = cv2.imread(image_path)

# カラー変換 (OpenCVはBGRなのでRGBに直す)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# バウンディングボックスを描画
for pred in prediction['predictions']:
    x = int(pred['x'])
    y = int(pred['y'])
    w = int(pred['width'])
    h = int(pred['height'])
    
    # バウンディングボックス座標計算
    x1 = x - w // 2
    y1 = y - h // 2
    x2 = x + w // 2
    y2 = y + h // 2

    # 色をクラスごとに決める（適当に固定色でもOK）
    color = (255, 0, 255)  # ピンク色（RGB）
    
    # 四角形を描く
    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

# 保存
output_path = "output_boxes_only.jpg"
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # 保存前にBGRに戻す
cv2.imwrite(output_path, img)

print(f"バウンディングボックスだけ描いた画像を保存しました！ → {output_path}")


loading Roboflow workspace...
loading Roboflow project...
バウンディングボックスだけ描いた画像を保存しました！ → output_boxes_only.jpg


In [19]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key="Bkn7HMszGVliBmChzz78"
)

result = CLIENT.infer("test2.jpg", model_id="structure-inspector/2")

# 🔥 ここを追加！
print(result)


{'inference_id': '9058530e-da54-4f12-88f6-246f34ecbd4c', 'time': 0.00755611000204226, 'image': {'width': 345, 'height': 230}, 'predictions': [{'x': 136.5, 'y': 122.0, 'width': 159.0, 'height': 158.0, 'confidence': 0.8781711459159851, 'class': 'Gusset Plate Connection', 'class_id': 2, 'detection_id': 'fa8f6f12-4223-4d17-a62e-04a94999a3aa'}]}


In [21]:
import cv2

# 推論結果 (さっきのresult)
# ここにresultを渡しておく必要あり！

# 元画像を読み込む
img = cv2.imread("test2.jpg")

# バウンディングボックス＋ラベルを描く
for pred in result['predictions']:
    x = int(pred['x'])
    y = int(pred['y'])
    w = int(pred['width'])
    h = int(pred['height'])
    
    # 左上座標と右下座標を計算
    x1 = int(x - w / 2)
    y1 = int(y - h / 2)
    x2 = int(x + w / 2)
    y2 = int(y + h / 2)
    
    # 四角を描画（色：ピンク、線幅：2）
    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 2)
    
    # ラベル名を描画（座標の少し上に）
    label = pred['class']
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_thickness = 1
    text_size, _ = cv2.getTextSize(label, font, font_scale, font_thickness)
    text_x = x1
    text_y = y1 - 5 if y1 - 5 > 10 else y1 + 15  # 枠外に出ないように調整

    # 背景用の塗りつぶし（オプション：見やすくする）
    cv2.rectangle(img, (text_x, text_y - text_size[1]), (text_x + text_size[0], text_y), (255, 0, 255), -1)
    
    # テキストを白文字で重ねる
    cv2.putText(img, label, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

# 画像保存
cv2.imwrite("output.jpg", img)

print("バウンディングボックス＋ラベルを描画して保存しました！")


バウンディングボックス＋ラベルを描画して保存しました！
